In [1]:
import json

import numpy as np

from src import create_wikipedia_corpus, vectorize_corpus

In [2]:
with open("data/wikipedia_corpus.json", "r", encoding="utf-8") as f:
    documents = json.load(f)

corpus, _ = vectorize_corpus(documents) # Vectorized documents
D = len(corpus) # Number of documents

In [ ]:
def SVI_for_LDA(n_iterations=100):
    for i in range(n_iterations):
        pass